In [21]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 2.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import openpyxl
import warnings


In [12]:
#ignoring the depreciation warnings that were flagging up 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#cycling through the list of files extracting the chagas data, renaming relevant columns such that they were consistent across the years. 
relevant_columns = ['DEPT_CODE', 'DEPT', 'MUN_CODE', 'MUN', 'TOTAL']
for i in range(12,22):
    if i >= 19: #needed as the different files all needed different things changed
        #note that I have already cleaned it up a little by hand to ensure that each excel only has the chagas data, rather than the many other diseases that were included 
        df = pd.read_excel('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_illness_cases/rutinaria_20' + str(i) + '_chagas_only.xlsx', skiprows = 1)
        if i == 21:
            df = df.rename(columns = {"Departamento_Ocurrencia": "NDEP_PROCE", 'Municipio_Ocurrencia': 'NMUN_PROCE', 'evento': 'NOM_EVE' })
        df = df[['COD_DPTO_O', 'NDEP_PROCE', 'COD_MUN_O', 'NMUN_PROCE', 'Grand Total']]
        df = df.rename(columns = {'COD_DPTO_O': 'DEPT_CODE', 'NDEP_PROCE': 'DEPT', 'COD_MUN_O': 'MUN_CODE', 'NMUN_PROCE': 'MUN',  'Grand Total':'TOTAL'})
        df = df.fillna(method='ffill') #needed as the data was presented such that the department name for each municipality (like the region for each county) was only written once.
        df = df[df["MUN"].str.contains('Total')==False]
        df = df[df["DEPT_CODE"].str.contains('Total')==False]
        df.to_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_chagas/colombia_20'+str(i) +'.csv', index = False)
        
    if i < 19:
        df = pd.read_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_illness_cases/rutinaria_20' + str(i) + '.csv')
        df = df.drop(['COD_EVE', 'SEMANA'], axis = 1)
        # print(i)
        if i in [13,14,16]:
            df = df.drop('semana_', axis = 1)
        elif i in [12,15]:
            df = df.drop('Semana_', axis = 1)
        elif i==17:
            df = df.drop('TITULO_SEMANA', axis = 1)
            df = df.rename(columns = {'cod_dep':'COD_DPTO','cod_mun':'Cod_municipio'})
            
        elif i==18 :
            df = df.rename(columns = {'Evento': 'Nombre','Cod_departamento': 'COD_DPTO', 'cod_municipio':'Cod_municipio'})
            df = df.drop('Titulo_semana', axis = 1)
            df = df.drop('mun', axis = 1)
            # print(df.columns)
        if i != 18:
            df = df.drop("ANO", axis = 1)
            # print(df.columns)
        df = df[df['Nombre'] == 'CHAGAS']
        df = df.drop('Nombre', axis = 1)
        df = df.rename(columns = {'COD_DPTO': 'DEPT_CODE', 'Departamento': 'DEPT', 'Cod_municipio': 'MUN_CODE', 'Municipio': 'MUN'})
        df_sum = pd.DataFrame(columns = relevant_columns)
        municipalities = df['MUN'].unique()
        # print(len(municipalities))
        for municipal in municipalities:
            df_municipality = df[df['MUN'] == municipal]
            #print(df_municipality)
            cases = df_municipality['conteo']
            total_cases = cases.sum()
            info = pd.Series(df_municipality.iloc[0])
            info['conteo'] = total_cases
            info = info.rename({'conteo':'TOTAL'})
            df_sum = df_sum.append(info, ignore_index= True)
        df_sum.to_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_chagas/colombia_20'+str(i) +'.csv', index = False)



/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3802507846.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append(info, ignore_index= True)
/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3802507846.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append(info, ignore_index= True)
/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3802507846.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append(info, ignore_index= True)
/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3802507846.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [13]:
#concatenating all the cleaned up data sets into one dataframe
df1 = pd.read_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_chagas/colombia_2012.csv')
df2 = pd.read_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_chagas/colombia_2013.csv')
df = pd.concat([df1, df2])
print(df)
for i in range(14,22):
    df_temp = pd.read_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_chagas/colombia_20'+str(i)+'.csv')
    df = pd.concat([df, df_temp])
    print(i)
    print(df.columns)


municipalities = df['MUN'].unique()

#finding the number of cases in each municipality
df_sum = pd.DataFrame(columns = relevant_columns)
for municipal in municipalities:
    df_municipality = df[df['MUN'] == municipal]
    cases = df_municipality['TOTAL']
    total_cases = cases.sum()
    info = pd.Series(df_municipality.iloc[0])
    info['TOTAL'] = total_cases
    df_sum = df_sum.append(info, ignore_index= True)

df_sum = df_sum[df_sum["MUN"].str.contains('DESCONOCIDO')==False] #removing the chagas cases of unknown origin
#df_sum.to_csv('/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/chagas_cases/colombia_chagas/colombia_total.csv', index = False)


     DEPT_CODE             DEPT  MUN_CODE                MUN  TOTAL
0           20            CESAR     20011          AGUACHICA     19
1           50             META     50001      VILLAVICENCIO     22
2           15           BOYACA     15299            GARAGOA      2
3           54  NORTE SANTANDER     54001             CUCUTA     26
4           54  NORTE SANTANDER     54874  VILLA DEL ROSARIO     10
..         ...              ...       ...                ...    ...
176         68        SANTANDER     68327             GUEPSA      1
177         18          CAQUETA     18479            MORELIA      1
178         54  NORTE SANTANDER     54820             TOLEDO      1
179          8        ATLANTICO      8758            SOLEDAD      1
180         85         CASANARE     85250     PAZ DE ARIPORO      1

[403 rows x 5 columns]
14
Index(['DEPT_CODE', 'DEPT', 'MUN_CODE', 'MUN', 'TOTAL'], dtype='object')
15
Index(['DEPT_CODE', 'DEPT', 'MUN_CODE', 'MUN', 'TOTAL'], dtype='object')
16
Index

/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3986404841.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append(info, ignore_index= True)
/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3986404841.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append(info, ignore_index= True)
/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3986404841.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append(info, ignore_index= True)
/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_8309/3986404841.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con